# Data Denoising

## Load Libraries

In [2]:
import pandas as pd
import re
from collections import defaultdict
from sklearn.model_selection import train_test_split
# from hanspell import spell_checker
from jamo import h2j, j2hcj
from g2pk import G2p

## Just Split

In [3]:
total_data = pd.read_csv('../data/train.csv')
total_data

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,3,IT과학,"{'annotators': ['08', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
1,ynat-v1_train_00001,어버이날 막따가 흐려저…남부지방 여튼 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
2,ynat-v1_train_00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
3,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회,2,IT과학,"{'annotators': ['14', '13', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00
4,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,사회,2,IT과학,"{'annotators': ['16', '11', '15'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.28. 오전 9:54
...,...,...,...,...,...,...,...,...
52525,ynat-v1_train_45673,사우디 아람코 주요 석유시설·유전 드론 공격에 화재,세계,4,세계,"{'annotators': ['14', '07', '08'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.09.14. 오후 4:36
52526,ynat-v1_train_45674,박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데,스포츠,6,스포츠,"{'annotators': ['09', '13', '18'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2019.11.03 17:06
52527,ynat-v1_train_45675,아시안게임 첫 경기 앞둔 라건아 인도네시아 팬이라도 ...,스포츠,6,스포츠,"{'annotators': ['07', '08', '12'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.08.13 17:54
52528,ynat-v1_train_45676,트럼프 미중 무역협상 1단계 합의 서명식 가질 것종합,세계,4,세계,"{'annotators': ['07', '18', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.25. 오후 4:36


In [4]:
# 베이스라인 코드와 동일하게 7:3 train_test_split
train_data, eval_data = train_test_split(total_data, train_size=0.7, random_state=42)

In [4]:
train_data.sort_index().head(10)

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,3,IT과학,"{'annotators': ['08', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
2,ynat-v1_train_00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
3,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회,2,IT과학,"{'annotators': ['14', '13', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00
5,ynat-v1_train_00004,회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간,생활문화,3,생활문화,"{'annotators': ['03', '02', '12'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.04.03. 오전 7:05
8,ynat-v1_train_00006,월드컵 태극전사 16강 전초기지 레오강 입성종합,스포츠,6,스포츠,"{'annotators': ['10', '02', '07'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.06.04 08:15
9,ynat-v1_train_00007,미세먼지 속 출근길,생활문화,3,사회,"{'annotators': ['03', '10', '09'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.17. 오전 10:14
10,ynat-v1_train_00007,미세먼지 속 출근길,생활문화,3,사회,"{'annotators': ['03', '10', '09'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.17. 오전 10:14
12,ynat-v1_train_00009,베트남 경제 고성장 지속…2분기 GDP 6.71% 성장,세계,4,세계,"{'annotators': ['14', '10', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.28. 오후 5:42
14,ynat-v1_train_00011,정진석 이정현 당현실 냉정하게 봐야…물러나는게 좋다종합,정치,0,정치,"{'annotators': ['08', '14', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.05. 오후 7:30
15,ynat-v1_train_00012,美 베네수엘라 구호품 반입 촉구 안보리 결의 추진,세계,4,세계,"{'annotators': ['18', '10', '12'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.02.27. 오전 8:21


In [5]:
# train_data.to_csv('../data/train_v0.csv', index=False)
# eval_data.to_csv('../data/eval_v0.csv', index=False)

## Drop Duplicates
- 모든 feature가 동일하게 중복되는 데이터만 제거
- G2P 및 labeling error 데이터는 제외(이후에 처리할 예정)

In [6]:
# 중복 데이터 확인
total_data[total_data.duplicated(keep=False)].sort_index()

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
9,ynat-v1_train_00007,미세먼지 속 출근길,생활문화,3,사회,"{'annotators': ['03', '10', '09'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.17. 오전 10:14
10,ynat-v1_train_00007,미세먼지 속 출근길,생활문화,3,사회,"{'annotators': ['03', '10', '09'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.17. 오전 10:14
76,ynat-v1_train_00070,강풍·영하권 날씨…오후부터 비까지 내려,생활문화,3,생활문화,"{'annotators': ['09', '07', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.19. 오전 5:51
77,ynat-v1_train_00070,강풍·영하권 날씨…오후부터 비까지 내려,생활문화,3,생활문화,"{'annotators': ['09', '07', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.19. 오전 5:51
546,ynat-v1_train_00476,레이커스 NBA 성탄 매치서 골든스테이트에 완승,스포츠,6,스포츠,"{'annotators': ['03', '10', '18'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.12.26 12:45
...,...,...,...,...,...,...,...,...
50834,ynat-v1_train_44197,정치권 신다당제로 재편…치열한 캐스팅보트 경쟁 예고,정치,0,정치,"{'annotators': ['16', '15', '08'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.02.13. 오후 12:05
50846,ynat-v1_train_44207,카메룬 경찰 반정부 시위 야당 지도자들 체포,세계,4,세계,"{'annotators': ['14', '03', '13'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.30. 오전 1:36
50847,ynat-v1_train_44207,카메룬 경찰 반정부 시위 야당 지도자들 체포,세계,4,세계,"{'annotators': ['14', '03', '13'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.30. 오전 1:36
52318,ynat-v1_train_45486,기자회견 하는 청년 전태일 회원들,사회,2,사회,"{'annotators': ['16', '07', '08'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.09.11. 오전 11:12


In [5]:
# 중복 데이터 제거 및 train_test_split 함수
def train_test_split_with_drop_duplicates(total_df: pd.DataFrame):
    """
    전체 데이터가 입력되었을 때 베이스라인 코드의 split 데이터를 유지하면서,
    모든 columns가 동일한 중복 데이터를 삭제하는 코드입니다.

    미리 split된 데이터에 각각 적용하기에는 전체 데이터셋 내의 중복 데이터가
    하나의 데이터셋 내에 둘 다 존재할지, 두 데이터셋에 나눠 존재할지 알기가 힘들기 때문에,
    과잉 제거를 방지하고자 전체 데이터셋을 입력으로 받고 내부에서 split해 반환합니다.

    Args:
        total_df (pd.DataFrame): pandas.read_csv로 읽어온 베이스라인 데이터를 split 없이 그대로 입력합니다.

    Returns:
        train_drop (pd.DataFrame), eval_drop (pd.DataFrame): 7:3 split된 채로 전체 데이터의 중복 데이터가 drop된 train_drop, eval_drop 데이터가 반환됩니다.
    """
    duplicated_indices = total_df[total_df.duplicated(keep='first')].index.tolist()
    train_df, eval_df = train_test_split(total_df, train_size=0.7, random_state=42)
    train_drop = train_df[~train_df.index.isin(duplicated_indices)]
    eval_drop = eval_df[~eval_df.index.isin(duplicated_indices)]

    return train_drop, eval_drop

In [6]:
train_drop_du, eval_drop_du = train_test_split_with_drop_duplicates(total_data)

print("Total dropped data:", len(total_data) - len(train_drop_du) - len(eval_drop_du))
print("Dropped train data", len(train_data) - len(train_drop_du))
print("Dropped eval data", len(eval_data) - len(eval_drop_du))

Total dropped data: 156
Dropped train data 103
Dropped eval data 53


In [9]:
train_drop_du.sort_index().head(10)

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,3,IT과학,"{'annotators': ['08', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
2,ynat-v1_train_00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
3,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회,2,IT과학,"{'annotators': ['14', '13', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00
5,ynat-v1_train_00004,회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간,생활문화,3,생활문화,"{'annotators': ['03', '02', '12'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.04.03. 오전 7:05
8,ynat-v1_train_00006,월드컵 태극전사 16강 전초기지 레오강 입성종합,스포츠,6,스포츠,"{'annotators': ['10', '02', '07'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.06.04 08:15
9,ynat-v1_train_00007,미세먼지 속 출근길,생활문화,3,사회,"{'annotators': ['03', '10', '09'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.17. 오전 10:14
12,ynat-v1_train_00009,베트남 경제 고성장 지속…2분기 GDP 6.71% 성장,세계,4,세계,"{'annotators': ['14', '10', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.28. 오후 5:42
14,ynat-v1_train_00011,정진석 이정현 당현실 냉정하게 봐야…물러나는게 좋다종합,정치,0,정치,"{'annotators': ['08', '14', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.05. 오후 7:30
15,ynat-v1_train_00012,美 베네수엘라 구호품 반입 촉구 안보리 결의 추진,세계,4,세계,"{'annotators': ['18', '10', '12'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.02.27. 오전 8:21
16,ynat-v1_train_00013,황재균 쐐기타…kt 갈 길 바쁜 삼성에 고춧가루,스포츠,6,스포츠,"{'annotators': ['16', '14', '07'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.09.16 17:28


In [10]:
# train_drop_du.to_csv('../data/train_v0.3.csv', index=False)
# eval_drop_du.to_csv('../data/eval_v0.3.csv', index=False)

## Drop Label Errors

- 모든 칼럼이 동일한 중복 데이터 제거 및 train-eval split 이후 적용(after train_test_split_with_drop_duplicates)
- Label을 의도적으로 틀리게 만든 데이터 제거

In [11]:
# category dict의 번호순으로 label 이름 정렬
categories = ['정치', '경제', '사회', '생활문화', '세계', 'IT과학', '스포츠']
cat2num = {cat: idx for idx, cat in enumerate(categories)}
num2cat = {idx: cat for idx, cat in enumerate(categories)}

def anno2df(df: pd.DataFrame):
    """
    데이터 df의 annotations 정보를 pd.DataFrame으로 변환한 annotations_df를 출렵합니다.

    Args:
        df (pd.DataFrame): annotations column이 존재하는 DataFrame을 입력 받습니다.

    Returns:
        annotations_df (pd.DataFrame): annotations 정보를 DataFrame으로 변환하여 출력합니다.
    """
    annotations_dict = defaultdict(list)
    for idx, d in df.iterrows():
        anno_info = eval(d.annotations)
        annos = anno_info['annotations']
        
        annotations_dict['annotators'].append(anno_info['annotators'])
        for anno_key, anno_values in annos.items():
            annotations_dict[anno_key].append(anno_values)

    annotations_df = pd.DataFrame.from_dict(annotations_dict)
    annotations_df

    return annotations_df

def get_labels_from_annotations(annotations_df: pd.DataFrame):
    """
    annotations_df의 scope 정보로부터 label_text와 target을 추론하여,
    label_text_from_annotstions와 target_from_annotations를 출력합니다.

    Args:
        annotations_df (pd.DataFrame): annotations 정보가 DataFrame 형태로 변환된 annotations_df를 입력 받습니다.

    Returns:
        label_text_from_annotations (List): annotations_df의 정보로부터 추측된 label_text를 출력합니다.
        target_from_annotations (List): annotations_df의 정보로부터 추측된 label_text의 target을 출력합니다.
    """
    label_text_from_annotations, target_from_annotations = [], []
    for idx, anno in annotations_df.iterrows():
        score1, score2, score3 = [0] * 7, [0] * 7, [0] * 7
        for s1, s2, s3 in zip(anno['first-scope'], anno['second-scope'], anno['third-scope']):
            if s1 != '해당없음':
                score1[cat2num[s1]] += 1
            if s2 != '해당없음':
                score2[cat2num[s2]] += 1
            if s3 != '해당없음':
                score3[cat2num[s3]] += 1
                
        if max(score1) > 1:
            score = score1
        else:
            print('first-scope 동점!')
            if max(score2) > 0:
                score = score2
            else:
                print('second-scope 동점!')
                score = score3

        label_argmax = max(range(len(score)), key=lambda x: score[x])
        target_from_annotations.append(label_argmax)
        label_text_from_annotations.append(num2cat[label_argmax])

    return label_text_from_annotations, target_from_annotations

In [12]:
label_from_annotations, _ = get_labels_from_annotations(anno2df(train_data))
label_from_annotations[:10]

['경제', '스포츠', '경제', '세계', '세계', '사회', '세계', '세계', '세계', '세계']

In [13]:
train_data[train_data.label_text != label_from_annotations]

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
51301,ynat-v1_train_44600,매서운 바람에 강추위…호남·제주 내일까지 눈,스포츠,6,생활문화,"{'annotators': ['13', '11', '09'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.22. 오전 6:00
4918,ynat-v1_train_04253,김동철 靑 정책기획委 대통령이 제왕되려는 것…백지화해야,생활문화,3,정치,"{'annotators': ['13', '10', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2017.10.13. 오전 9:48
43292,ynat-v1_train_37654,의원발언 경청하는 이낙연 총리,생활문화,3,정치,"{'annotators': ['11', '12', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.05.24. 오전 10:41
17807,ynat-v1_train_15501,선거구 획정위 오후 전체회의서 획정안 도출 시도,IT과학,5,정치,"{'annotators': ['03', '09', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.02.27. 오전 9:09
24266,ynat-v1_train_21132,제임스 이탈한 레이커스 새크라멘토에 1점 차 패배,사회,2,스포츠,"{'annotators': ['09', '12', '10'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.12.28 15:49
...,...,...,...,...,...,...,...,...
16646,ynat-v1_train_14493,北 3월부터 주민들 중국 개별여행 불허 RFA,사회,2,정치,"{'annotators': ['03', '02', '11'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.16. 오전 8:49
21834,ynat-v1_train_19036,펜스 미 부통령 차량 금지 섬에 들어갔다 구설,IT과학,5,세계,"{'annotators': ['03', '07', '11'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.09.23. 오전 10:46
11534,ynat-v1_train_10016,KT 수능 수험생에 6개월간 데이터 무제한 요금제 할인,생활문화,3,IT과학,"{'annotators': ['18', '10', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2017.11.02. 오전 9:50
11016,ynat-v1_train_09561,그래픽 축구대표팀 월드컵 유니폼 공개,정치,0,스포츠,"{'annotators': ['03', '14', '13'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.03.22 14:35


In [14]:
def drop_label_errors(df_with_label_text):
    """
    label_text와 annotations column을 포함하는 데이터에서,
    label_text와 label_text_from_annotations가 다른 경우 label error로 판단하여 제거합니다.

    Args:
        df_with_label_text (pd.DataFrame): label_text와 annotations column이 존재하는 데이터를 입력받습니다.

    Returns:
        df_with_label_text (pd.DataFrame): label error가 제거된 데이터를 반환합니다.
    """
    labels_from_annotations, _ = get_labels_from_annotations(anno2df(df_with_label_text))
    label_error_data = df_with_label_text[df_with_label_text.label_text != labels_from_annotations]
    print("Label Error Data: {}".format(len(label_error_data)))

    df_with_label_text = df_with_label_text.drop(index=label_error_data.index)

    return df_with_label_text

In [15]:
train_drop_du_le = drop_label_errors(train_drop_du)
print("{} label error train data dropped".format(len(train_drop_du) - len(train_drop_du_le)))

Label Error Data: 981
981 label error train data dropped


In [16]:
eval_drop_du_le = drop_label_errors(eval_drop_du)
print("{} label error eval data dropped".format(len(eval_drop_du) - len(eval_drop_du_le)))

Label Error Data: 390
390 label error eval data dropped


In [17]:
# train_drop_du_le.to_csv('../data/train_v0.4.csv', index=False)
# eval_drop_du_le.to_csv('../data/eval_v0.4.csv', index=False)

## Drop Wrong Spelling (G2P) Errors

- 모든 칼럼이 동일한 중복 데이터 제거 및 train-eval split 이후 적용(after train_test_split_with_drop_duplicates)
- 맞춤법이 G2P 방식(발음대로)으로 noising된 데이터들을 제거

In [65]:
sample_text1 = "애플의"
print(j2hcj(h2j(sample_text1)), " length:", len(j2hcj(h2j(sample_text1))))

sample_text2 = "애프릐"
print(j2hcj(h2j(sample_text2)), " length:", len(j2hcj(h2j(sample_text2))))

ㅇㅐㅍㅡㄹㅇㅢ  length: 7
ㅇㅐㅍㅡㄹㅢ  length: 6


In [45]:
g2p = G2p()
sent = "유은헤 부총리 일제 식민지 피해 실태와 과제 심포지엄 참석"
print(g2p(sent, descriptive=False))
print(g2p(sent, descriptive=True))
g2p_sent = "유은헤 부총니 일제 싱민지 피해 실태와 과제 심포지엄 참석"
print(g2p(sent, descriptive=True) == g2p_sent)

유은헤 부총니 일제 싱민지 피해 실태와 과제 심포지엄 참석
유은헤 부총니 일제 싱민지 피해 실태와 과제 심포지엄 참석
True


In [3]:
def drop_g2p_errors(df: pd.DataFrame):
    """
    데이터 내 G2P 에러 데이터를 제거합니다.
    원본과 비교했을 때,
    (1) 숫자가 사라졌는가(한글로 바뀌었는가)
    (2) 영어가 사라졌는가(한글로 바뀌었는가)
    (3) 받침이 사라졌는가(전체 자모 개수가 줄었는가)
    (4) 어절의 초성에 쌍자음이 생겼는가
    의 기준으로 phoneme 데이터를 판단하며, 위 과정 이후 남은 G2P 데이터는
    (5) G2P 변환기(g2pk.G2p())를 이용하여, ID가 동일한 두 문장 중 한 문장을 G2P 변환할 시 다른 하나로 변경되는지의 여부로 문장을 판별합니다.

    Args:
        df (pd.DataFrame): DataFrame 형태의 데이터를 입력 받습니다.

    Returns:
        df_ge (df.DataFrame): G2P 에러가 제거된 DataFrame 형태의 데이터를 출력합니다.
    """
    errors = df[df.duplicated(subset='ID', keep=False)].sort_index()
    errors_copy = errors.copy()

    g2p = G2p()
    error_indices = []
    for (idx1, e1), (idx2, e2) in zip(errors_copy.iloc[:-1:2].iterrows(), errors_copy.iloc[1::2].iterrows()):
        if e1.input_text == e2.input_text:
            continue
        
        input_words1, input_words2 = e1.input_text.split(), e2.input_text.split()

        error_idx = None
        for iw1, iw2 in zip(input_words1, input_words2):
            jamo_iw1, jamo_iw2 = j2hcj(h2j(iw1)), j2hcj(h2j(iw2))

            # 숫자가 사라졌는가
            numeric_pattern = r"\d"
            if not re.search(numeric_pattern, iw1) and re.search(numeric_pattern, iw2):
                error_idx = idx1
                break
            elif re.search(numeric_pattern, iw1) and not re.search(numeric_pattern, iw2):
                error_idx = idx2
                break

            # 영어가 사라졌는가
            alphabet_pattern = r"[a-zA-Z]"
            if not re.search(alphabet_pattern, iw1) and re.search(alphabet_pattern, iw2):
                error_idx = idx1
                break
            elif re.search(alphabet_pattern, iw1) and not re.search(alphabet_pattern, iw2):
                error_idx = idx2
                break

            # 받침이 사라졌는가
            len_iw1, len_iw2 = len(jamo_iw1), len(jamo_iw2)
            if len_iw1 < len_iw2:
                error_idx = idx1
                break
            elif len_iw1 > len_iw2:
                error_idx = idx2
                break

            # 쌍자음이 생겨났는가
            for single_iw1, single_iw2 in zip(list(iw1)[:min(len(iw1), len(iw2))], list(iw2)[:min(len(iw1), len(iw2))]):
                jamo_single_iw1, jamo_single_iw2 = j2hcj(h2j(single_iw1)), j2hcj(h2j(single_iw2))
                double_pattern = r"ㄲ|ㄸ|ㅃ|ㅆ|ㅉ"
                if re.search(double_pattern, jamo_single_iw1[0]) and not re.search(double_pattern, jamo_single_iw2[0]):
                    error_idx = idx1
                    break
                elif not re.search(double_pattern, jamo_single_iw1[0]) and re.search(double_pattern, jamo_single_iw2[0]):
                    error_idx = idx2
                    break
            if error_idx:
                break
        
        # 남은 문장들에 대해 G2P 변환기를 써서 일치하는 지 확인
        if not error_idx:
            if e2.input_text == g2p(e1.input_text, descriptive=False) or e2.input_text == g2p(e1.input_text, descriptive=True):
                error_idx = idx1
            elif e1.input_text == g2p(e2.input_text, descriptive=False) or e1.input_text == g2p(e2.input_text, descriptive=True):
                error_idx = idx2

        if error_idx:
            error_indices.append(error_idx)
            error_idx = None

    print("G2P Removed:", len(error_indices)) # G2P 에러로 판단되어 제거된 에러의 개수

    df_ge = df.drop(index=error_indices)

    return df_ge

In [4]:
print(drop_g2p_errors.__doc__)


    데이터 내 G2P 에러 데이터를 제거합니다.
    원본과 비교했을 때,
    (1) 숫자가 사라졌는가(한글로 바뀌었는가)
    (2) 영어가 사라졌는가(한글로 바뀌었는가)
    (3) 받침이 사라졌는가(전체 자모 개수가 줄었는가)
    (4) 어절의 초성에 쌍자음이 생겼는가
    의 기준으로 phoneme 데이터를 판단하며, 위 과정 이후 남은 G2P 데이터는
    (5) G2P 변환기(g2pk.G2p())를 이용하여, ID가 동일한 두 문장 중 한 문장을 G2P 변환할 시 다른 하나로 변경되는지의 여부로 문장을 판별합니다.

    Args:
        df (pd.DataFrame): DataFrame 형태의 데이터를 입력 받습니다.

    Returns:
        df_ge (df.DataFrame): G2P 에러가 제거된 DataFrame 형태의 데이터를 출력합니다.
    


In [43]:
train_drop_du_ge = drop_g2p_errors(train_drop_du)
print(len(train_drop_du_ge))
# train_drop_du_ge.to_csv('../data/train_v0.5.csv', index=False)

G2P Removed: 2600
34068


In [44]:
eval_drop_du_ge = drop_g2p_errors(eval_drop_du)
print(len(eval_drop_du_ge))
# eval_drop_du_ge.to_csv('../data/eval_v0.5.csv', index=False)

G2P Removed: 499
15207


In [ ]:
# errors.to_csv('../data/errors_clean_g2p_complete.csv', index=True)

# Make Test File's Answers (Confidential)

In [26]:
# category dict의 번호순으로 label 이름 정렬
categories = ['정치', '경제', '사회', '생활문화', '세계', 'IT과학', '스포츠']
cat2num = {cat: idx for idx, cat in enumerate(categories)}
num2cat = {idx: cat for idx, cat in enumerate(categories)}
cat2num

{'정치': 0, '경제': 1, '사회': 2, '생활문화': 3, '세계': 4, 'IT과학': 5, '스포츠': 6}

In [119]:
def anno2df(df: pd.DataFrame):
    annotations_dict = defaultdict(list)
    for idx, d in df.iterrows():
        anno_info = eval(d.annotations)
        annos = anno_info['annotations']
        
        annotations_dict['annotators'].append(anno_info['annotators'])
        for anno_key, anno_values in annos.items():
            annotations_dict[anno_key].append(anno_values)

    annotations_df = pd.DataFrame.from_dict(annotations_dict)
    annotations_df

    return annotations_df

In [121]:
train_annotations = anno2df(train_data)
train_annotations

,annotators,first-scope,second-scope,third-scope
0,"[12, 09, 11]","[경제, 경제, 경제]","[해당없음, 해당없음, 해당없음]","[해당없음, 해당없음, 해당없음]"
1,"[12, 14, 09]","[스포츠, 스포츠, 스포츠]","[해당없음, 해당없음, 해당없음]","[해당없음, 해당없음, 해당없음]"
2,"[02, 12, 11]","[사회, 경제, 경제]","[경제, IT과학, 사회]","[해당없음, 해당없음, IT과학]"
3,"[16, 08, 07]","[세계, 세계, 세계]","[사회, 해당없음, 정치]","[해당없음, 해당없음, 해당없음]"
4,"[02, 10, 14]","[세계, 세계, 세계]","[사회, 해당없음, 정치]","[IT과학, 해당없음, 사회]"
...,...,...,...,...
36766,"[09, 07, 18]","[생활문화, 생활문화, 생활문화]","[해당없음, 해당없음, 해당없음]","[해당없음, 해당없음, 해당없음]"
36767,"[14, 02, 03]","[세계, 세계, 세계]","[사회, 사회, 사회]","[해당없음, 해당없음, 해당없음]"
36768,"[13, 02, 15]","[경제, 경제, 경제]","[해당없음, 해당없음, 사회]","[해당없음, 해당없음, 해당없음]"
36769,"[10, 15, 13]","[IT과학, 경제, IT과학]","[해당없음, 사회, 해당없음]","[해당없음, IT과학, 해당없음]"


In [131]:
def get_labels_from_annotations(annotations_df: pd.DataFrame):
    label_from_annotations, target_from_annotations = [], []
    for idx, anno in annotations_df.iterrows():
        score1, score2, score3 = [0] * 7, [0] * 7, [0] * 7
        for s1, s2, s3 in zip(anno['first-scope'], anno['second-scope'], anno['third-scope']):
            if s1 != '해당없음':
                score1[cat2num[s1]] += 1
            if s2 != '해당없음':
                score2[cat2num[s2]] += 1
            if s3 != '해당없음':
                score3[cat2num[s3]] += 1
                
        if max(score1) > 1:
            score = score1
        else:
            print('first-scope 동점!')
            if max(score2) > 0:
                score = score2
            else:
                print('second-scope 동점!')
                score = score3

        label_argmax = max(range(len(score)), key=lambda x: score[x])
        target_from_annotations.append(label_argmax)
        label_from_annotations.append(num2cat[label_argmax])

    return label_from_annotations, target_from_annotations

In [134]:
sum(get_labels_from_annotations(anno2df(train_data))[0] == train_data.label_text) / len(train_data)

0.9733213673819042

In [145]:
test_data = pd.read_csv('../data/test.csv')
label_text_from_annotations, target_from_annotations = get_labels_from_annotations(anno2df(test_data))
label_text_from_annotations


['사회',
 '사회',
 'IT과학',
 '경제',
 '사회',
 '사회',
 '생활문화',
 '사회',
 '사회',
 '경제',
 '사회',
 '사회',
 '경제',
 '사회',
 '정치',
 '경제',
 '경제',
 '생활문화',
 '사회',
 '사회',
 '생활문화',
 '경제',
 '사회',
 '사회',
 '경제',
 '생활문화',
 '사회',
 '사회',
 '사회',
 '생활문화',
 '경제',
 '경제',
 '경제',
 '생활문화',
 '사회',
 '경제',
 '생활문화',
 '사회',
 '경제',
 '사회',
 '사회',
 '사회',
 '세계',
 '사회',
 '사회',
 '스포츠',
 '세계',
 '사회',
 'IT과학',
 '사회',
 '정치',
 '세계',
 '사회',
 '스포츠',
 '정치',
 '사회',
 '생활문화',
 '스포츠',
 '정치',
 '경제',
 '사회',
 '스포츠',
 '사회',
 '정치',
 '사회',
 '사회',
 '세계',
 '사회',
 '세계',
 '경제',
 '생활문화',
 '경제',
 '생활문화',
 '사회',
 '사회',
 '세계',
 '생활문화',
 '스포츠',
 '경제',
 '스포츠',
 '경제',
 '경제',
 '경제',
 '사회',
 '사회',
 '생활문화',
 '사회',
 '세계',
 '스포츠',
 '경제',
 '세계',
 '경제',
 '사회',
 '사회',
 '사회',
 '세계',
 '사회',
 '경제',
 '정치',
 '경제',
 '사회',
 '사회',
 '경제',
 '사회',
 '사회',
 '사회',
 '스포츠',
 '정치',
 '경제',
 'IT과학',
 '사회',
 '생활문화',
 '정치',
 '경제',
 '생활문화',
 '사회',
 '세계',
 '생활문화',
 '스포츠',
 '사회',
 '생활문화',
 '정치',
 '생활문화',
 '사회',
 '사회',
 '경제',
 '사회',
 '스포츠',
 '사회',
 '경제',
 '경제',
 '경제',
 '사회',
 '스포츠',
 '경제',
 '경제

In [146]:
test_data.insert(2, 'label_text_from_annotations', label_text_from_annotations)
test_data.insert(3, 'target_from_annotations', target_from_annotations)
test_data.to_csv('../data/test_with_answers.csv', index=False)